<div align="right">Revision 1 : September 2022</div>

<a href="https://colab.research.google.com/github/dewdotninja/control_python/blob/master/exercises/loopshaping_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="left">
<img src="https://raw.githubusercontent.com/dewdotninja/exams/main/feng_heading_en.png" width=400 alt="Feng heading"/>
</p>

#### Department of Mechanical Engineering

### Transfer functions, frequency responses, feedback properties, and loopshaping design exercise


In [ ]:
# You need to install these libraries in the Colab environment
!pip install control

In [1]:
# import libraries you want to use
import numpy as np
import matplotlib.pyplot as plt
import control as ctl

**Note :** Some parameters in this notebook depends on random variables <code>rand_x, rand_y, rand_z</code> below. This makes 
the exercise different each time you run it.

In [2]:
rand_x = np.random.randint(1,9)
rand_y = np.random.randint(1,9)
rand_z = np.random.randint(1,9)

### P1 (10 points)

One device commonly used in industrial machines with rotary motion, such as robotic joints, is called a Harmonic Drive from a group of companies Harmonic Drive LLC. motor Hereafter, for the convenience of writing, it is abbreviated as the HDM system.

Figure 1 shows the structure of the HDM mechanism consisting of three parts: a circular spline, a flexspline, and an elliptical wave generator. The wave generator is connected to the shaft of a motor that rotates at high speed. A round ring has teeth inside. Between the two are separated by an elastic ring with teeth on the outside.

<p align="center">
<img src="https://drive.google.com/uc?id=1PtcGtTJ2iv81nLHz5Gb5mNnGeM_d5i4c" width=300 alt="Figure 1"/>
</p>
<div align="center"><b>Figure 1 HDM structure (https://www.harmonicdrive.net/)</b></div>

When drawing a schematic diagram of the HDM device connected to the DC motor, it will be shown in Figure 2. Left side is the motor's electrical system. and on the right is the mechanical system of the harmonic drive, represented by a soft shaft thru-axle gearbox with a hardness of k.

<p align="center">
<img src="https://drive.google.com/uc?id=1OP3LsMAgsHQ5uYdmVLQFHQLq55J-kYd7" width=550 alt="Figure 2"/>
</p>
<div align="center"><b>Figure 2 HDM block diagram </b></div>

Define
$$
p_l(s) = J_ls^2 + B_ls + k \tag{1}
$$
$$
p_m(s) = J_ms^2 + B_ms + k \tag{2}
$$
Overall transfer function of HDM can be described as 
$$
P(s) = \frac{\theta_l(s)}{V(s)} = \frac{k_mk}{p_m(s)p_l(s)(Ls+R)-k^2(Ls+R)+rk_mk_bsp_l(s)} \tag{3}
$$
assign parameter values (with x, y, z from your student ID)

```python
km = (rand_y+rand_z)*100  # torque constant
kb = 1  # back EMF constant
k = 1000 # torsional stiffness of harmonic drive
r = rand_x   # gear ratio
R = 1  # armature resistance
L = 0.1 # armature inductance
Jm = (rand_x+rand_y)  #  motor inertia
Bm = 0.01*rand_y  # motor shaft friction
Jl = (rand_x+rand_z)  # load inertia
Bl = 0.01*rand_z  # load friction
```

**P1.1** Write code to create a plant transfer function of HDM as in (3) (2 points)

#### Solution


**P1.2** Plot poles and zeros of the transfer function from  **P1.1** on complex plane (2 points)

#### Solution

**P 1.3** Show Bode plot of the plant from **P1.1** with magnitude in dB and phase in degrees, in frequency range
0.01 - 1000 rad/s. What are the gain and phase margins of this plant? (2 point) 
 
#### Solution

**P1.4** From the feedback block diagram in Figure 3, use the plant from **P1.1** and a proportional 
control <code>C(s) = K = 0.1*(rand_x+rand_y)</code>. Construct a transfer function mapping $d_i(s)$ to $y(s)$. If the closed-loop system 
is stable, find magnitude of $y(s)$ when $d_i(s) = sin(0.1t)$. Otherwise, simply state that the closed-loop 
system is unstable. 
(4 points)

<p align="center">
<img src="https://drive.google.com/uc?id=1CLNYYhbhGOpVaJULHdX13xoEv_AO2wyR" width=550 alt="Figure 3"/>
</p>
<div align="center"><b>Figure 3 general block diagram of feedback system</b></div>

#### Solution

***
#### P2 (10 points)

Apply loopshaping control design for the HDM plant above with the following specifications 

1. zero steady-state tracking error

2. Output disturbance is attenuated as least <code>0.01(rand_y)</code> for frequency below <code>0.1(rand_x)</code> rad/s

3. From the Bode plot, we observe 2 pairs of lightly-damped poles in high frequency region. Focus on the lower frequency pair. System bandwidth must be limited to at least 1 decade below the lightly-damped frequency mode. For instance, suppose that the lightly-damped mode frequency is 17 rad/s, the loop bandwidth (roughly the frequency where $|L(j\omega)|$ crosses the 0 dB line) must not exceed 1.7 rad/s. So, for this problem the high frequency criterion is not specified as noise attenuation, but instead a bandwidth limitation.

4. Closed-loop stable, with phase margin of at least 30 degrees.

**Remarks :**

* Lightly-damped mode frequency $\omega_d = \omega_n\sqrt{1-\zeta^2}$ can be found from the (absolute value of) imaginery part of the corresponding pair of lightly-damped poles.
* If you don't know how to specify high frequency criteria, just make the high frequency region starting from the lighly-damped mode frequency, and some small noise attenuation, say, -1 dB. The requirement here is bandwidth limitation. 

For your convenience, lshape() and plot_response() are provided in the following cell.

In [ ]:
def lshape(C,P, lf, lfb, hf, hfb, pm ):
    assert lf > 0   # avoid bad values
    assert lfb > 0
    assert hf > lf
    assert hfb < 0
    assert 0 < pm < 90
    L = C*P # form loop transfer function
    # create a suitable range of frequency from lf, hf
    lf_log10 = np.log10(lf)
    w_start = np.floor(lf_log10)-1
    hf_log10 = np.log10(hf)
    w_end = np.ceil(hf_log10)+1
    w = np.logspace(w_start,w_end, 1000)
    
    # frequency response of L
    Lmag, Lph, om = ctl.freqresp(L, w)
    Lmag_db = np.squeeze(20*np.log10(Lmag))
    Lph_deg = np.squeeze(np.degrees(Lph))
    
    # create bound vectors
    lf_mask = np.where(om<lf, lfb, 0)
    hf_mask = np.where(om<hf, 0, hfb)
    lf_bnd = lf_mask*np.ones(om.shape)
    hf_bnd = hf_mask*np.ones(om.shape)
    
    # check whether violation occurs
    lf_idxv = np.where(om>lf)
    lf_idx = lf_idxv[0][0]   # find index of low-freq region
    hf_idxv = np.where(om<hf)
    hf_idx = hf_idxv[0][-1]  # find index of high-freq region
    lfmag = Lmag_db[:lf_idx]
    hfmag = Lmag_db[hf_idx:]
    if min(lfmag)<lfb:
        lf_legend = "LF bound (violated)"
    else:
        lf_legend = "LF bound"
    if max(hfmag)>hfb:
        hf_legend = "HF bound (violated)"
    else:
        hf_legend = "HF bound"    
    
    # desired phase margin vectors
    pmvec = (pm-180)*np.ones(om.shape)
    
    # compute gain/phase margins
    g_margin, ph_margin, wgm, wpm = ctl.margin(L)
    ph_at_crossover = (ph_margin-180)
    # Loopshaping plot
    fig, (ax1, ax2) = plt.subplots(2, figsize=(8,8))
    fig.suptitle('$L(j\omega)$ v.s. bounds')
    ax1.semilogx(om, Lmag_db,'k-', om, lf_bnd, 'm-.', om, hf_bnd,'b-.')
    ax1.legend(["$|L(j\omega)|$",lf_legend,hf_legend],loc="lower left")
    ax1.grid(True)
    ax1.set_ylabel('magnitude (dB)')
    
    ax2.semilogx(om, Lph_deg,'k-',om, pmvec,'b-', wpm, ph_at_crossover,'r*')
    if ph_margin > pm:
        pmtext = "phase margin = " + str(round(ph_margin)) + " degrees"
    else:
        pmtext = "phase margin = " + str(round(ph_margin)) + " degrees (violated)"
        
    ax2.text(wpm,ph_at_crossover,pmtext)
    
    dpmtext = "Desired PM (" + str(pm) + " degrees)"
    ax2.set_xlabel('frequency (rad/s)')
    ax2.set_ylabel('phase (deg)')
    ax2.legend(["$\measuredangle L(j\omega)$",dpmtext],loc="lower left")
    ax2.grid(True)    
    
    # plot magnitude of S and T v.s bounds
    S = 1/(1+L)
    T = L/(1+L)
    
    # frequency responses of S and T
    Smag, Sph, om = ctl.freqresp(S, w)
    Tmag, Tph, om = ctl.freqresp(T, w)
    
    Smag_db = np.squeeze(20*np.log10(Smag))
    Tmag_db = np.squeeze(20*np.log10(Tmag))
   
    # check whether violation occurs
    lf_idxv = np.where(om>lf)
    lf_idx = lf_idxv[0][0]   # find index of low-freq region
    hf_idxv = np.where(om<hf)
    hf_idx = hf_idxv[0][-1]  # find index of high-freq region
    lfSmag = Smag_db[:lf_idx]
    hfTmag = Tmag_db[hf_idx:]
    if max(lfSmag)>-lfb:
        lf_legend = "LF bound (violated)"
    else:
        lf_legend = "LF bound"
    if max(hfTmag)>hfb:
        hf_legend = "HF bound (violated)"
    else:
        hf_legend = "HF bound"    
        
    # create data vector for stability bound in mid freq region
    
    om_mid = om[lf_idx:hf_idx]
    pm_r = np.radians(pm)
    x = np.sin(0.5*(np.pi - pm_r))/(np.sin(pm_r)) 
    x = 20*np.log10(x)
    bnds_mid = x*np.ones(om_mid.shape)
    
    ST_peak = max(max(Smag_db),max(Tmag_db))
    if ST_peak > x:
        mf_legend = "Stability bound (violated)"
    else:
        mf_legend = "Stability bound"
    
    plt.figure(figsize=(8,4))
    plt.semilogx(om,Smag_db,'k-',om,Tmag_db,'g-',om,-lf_bnd,'m-.',om,hf_bnd,'b-.',om_mid,bnds_mid,'r-.')
    plt.xlabel('frequency (rad/s)')
    plt.ylabel('magnitude (dB)')
    plt.legend(["$|S(j\omega)|$","$|T(j\omega)|$",lf_legend,hf_legend, mf_legend])
    plt.grid(True)
    plt.title('$|S(j\omega)|$ and $|T(j\omega)|$ v.s. bounds')
    plt.show()


In [ ]:
def plot_response(sys,u,t,title):
    tout, y = ctl.forced_response(sys,t, u)
    truncated_idx = 150  # get rid of transient
    tout = tout[truncated_idx:]
    u = u[truncated_idx:]
    y = y[truncated_idx:]
    fig, (ax1, ax2) = plt.subplots(2, figsize=(8,8))
    fig.suptitle(title)
    ax1.plot(tout,u,'b-')
    ax1.grid(True)
    ax1.set_ylabel('input')
    
    ax2.plot(tout,y,'r-')
    ax2.grid(True)
    ax2.set_ylabel('output')
    ax2.set_xlabel('time (sec)')

    plt.show()    

#### Grade rubric

1. Obtain $C(s)$ and show that the Bode plot of $L(s)$ satisfies all criteria. Minor violations at frequeny boundaries may be tolerated. Bandwidth of $L(s)$ should not exceen the tolerance of 0.15 of the first lightly-damped frequency (5 points)
2. Show step response that verifies closed-loop stablity and zero steady-state tracking error. (3 points)
3. Disturbance response from $d_o(s)$ meets the specification, with tolerance of 5% exceeding the attenuation criteria. (2 points)

#### Solution